In [1]:
import numpy as np
import pandas as pd 

import torch
torch.manual_seed(42)

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('X_test.csv')
submit = pd.read_csv('sample_submit.csv')

In [3]:
train

,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET
0,0,0,0,0,0,1.5,69.08,-12,0.000000
1,0,0,30,0,0,1.5,69.06,-12,0.000000
2,0,1,0,0,0,1.6,71.78,-12,0.000000
3,0,1,30,0,0,1.6,71.75,-12,0.000000
4,0,2,0,0,0,1.6,75.20,-12,0.000000
...,...,...,...,...,...,...,...,...,...
9994,208,5,0,0,0,0.5,64.12,12,0.000000
9995,208,5,30,26,383,0.8,56.18,13,5.630068
9996,208,6,0,41,578,1.1,47.46,15,13.887196
9997,208,6,30,52,699,1.4,44.51,17,23.269925


In [4]:
train_x = train.drop(['Day', 'TARGET'], axis = 1)
train_y = pd.DataFrame(train.TARGET)

test_x = test.drop(['ID','Day'],axis =1)

In [5]:
train_x_tensor = torch.FloatTensor(train_x.values)
train_y_tensor = torch.FloatTensor(train_y.values)

test_x_tensor = torch.FloatTensor(test_x.values)

In [6]:
linear1 = torch.nn.Linear(7,49, bias = True)
linear2 = torch.nn.Linear(49,49, bias = True)
linear3 = torch.nn.Linear(49,49, bias = True)
linear4 = torch.nn.Linear(49,1, bias = True)
relu = torch.nn.ReLU()

In [7]:
model = torch.nn.Sequential(linear1, relu, linear2, relu, linear3, relu, linear4)

In [8]:
loss = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr= 0.01 )

In [9]:
total_epoch = 1000

for epoch in range(1, total_epoch + 1):
    
    optimizer.zero_grad()
    hypothesis = model(train_x_tensor)
    
    cost = loss(hypothesis, train_y_tensor)
    cost.backward()
    
    optimizer.step()
    
    if epoch % 100 == 0:
        print(epoch, cost.item())

100 15.394104957580566
200 14.022342681884766
300 12.765750885009766
400 12.38613510131836
500 13.395434379577637
600 16.0253849029541
700 10.87927532196045
800 10.324292182922363
900 8.459189414978027
1000 7.201025009155273


In [10]:
train_hypothesis = model(train_x_tensor)
train_score = torch.mean((train_hypothesis - train_y_tensor)**2)
train_score = torch.sqrt(train_score)
print(train_score)

tensor(2.7258, grad_fn=<SqrtBackward>)


In [11]:
predict = model(test_x_tensor)
submit.TARGET = predict.detach().numpy()

In [12]:
submit.to_csv('2021_10_11_01.csv',index = False)